# Scraping af FT

In [1]:
from bs4 import BeautifulSoup as bs
import re 
import os
import lxml
import xml.etree.ElementTree as ET
from lxml import objectify
from datetime import datetime
import numpy as np 
import pandas as pd 
import requests
import ftplib
from io import BytesIO
import tqdm

##### Tomme lister

In [2]:
dirs = []
sub_dir = []
files = []
file_name = []
paths = []
raw_xml_list = []

suppe = []

### Login til ftp serveren og henter en liste over biblioteker

In [3]:
ftp_url = "oda.ft.dk"
ftp_dir = "/ODAXML/Referat/samling/"

In [4]:
ftp = ftplib.FTP(ftp_url)
ftp.login("anonymous", "wpg345@alumni.ku.dk")
ftp.set_pasv(True)
ftp.cwd(ftp_dir)
ftp.dir(dirs.append)

In [5]:
for i in range (0, len(dirs), 1):    # bygger en liste med underbiblioteker
    sub_dir.append(ftp_dir+dirs[i][-5:]+'/') 
    
for i in range (0, len(sub_dir), 1): # bygger en liste med filnavne
    ftp.cwd(sub_dir[i])
    ftp.dir(files.append)

for i in range(0, len(files)):       # extract filnavn
    file_name.append(files[i][39:])   

for i in range(0, len(file_name)):   # opbygning af den fulde sti og filnavn
    paths.append(ftp_dir+file_name[i][0:5]+'/'+file_name[i])

### Overførsel af de enkelte filer fra ftp://oda.ft.dk, og raw_xml skrives til liste

In [6]:
for i in tqdm.tqdm(range(0, len(paths))):
    r = BytesIO()
    ftp.retrbinary(f"RETR {paths[i]}", r.write)
    raw_xml_list.append(r.getvalue())
    r.close()

100%|██████████████████████████████████████████████████████████████████████████████| 1489/1489 [05:42<00:00,  4.35it/s]


# Define path function

In [9]:
# extract_path VIRKER
repeats = []

def extract_path(xml_list):
    
    for xml in xml_list: 
        soup = bs(xml, 'lxml')
        repeats.append(len(soup.find_all("dagsordenpunkt")))
    
    for i in range(0, len(xml_list)): 
        p = paths[i]
        path.extend([p] * repeats[i]) 
        #extend acts as append but multiplied by n times
    
    return None

Bemærk vi er interesseret i len(xml_list) modsat de næste funktioner, hvor det ikke behøver spille nogen rolle. Derfor vælger vi ikke at integrere extract_path ind i extract_metadata. 

# Define metadata function

In [10]:
# vers1 virker
def extract_metadata(xml):
    soup = bs(xml, 'lxml')
    
    d = soup.find("dateofsitting").text[:10]
    m = soup.find("meetingnumber").text
    s = soup.find("parliamentarysession").text
    g = soup.find("parliamentarygroup").text
    l = soup.find("location").text
    
    # antal gange de samme metadata gentages for hvert dagordenpunkt
    repeats = len(soup.find_all("dagsordenpunkt")) 
    
    for repeat in range(0, repeats):
        date.append(d)
        meetingnumber.append(m)
        session.append(s)
        group.append(g)
        location.append(l)
    
    return None

# Define content function 

In [11]:
def extract_content(xml): 
    soup = bs(xml, 'lxml')
    dagsordenpunkter = soup.find_all("dagsordenpunkt") # liste med dagsordenpunkter
    #print(f"Antal dagsordenpunkter: {len(dagsordenpunkter)}")
    
    for dp in dagsordenpunkter: # find raw xml-sætninger for hvert dagsordenpunkt
        sentences_raw = dp.find_all("linea") # liste med raw sætninger 
        #print(f"Antal sentences_raw: {len(sentences_raw)}")
        
        sentences_text = []
        
        for sr in sentences_raw: # for hver raw sætning, kør .text af sætningen
            sentences_text.append(sr.get_text(separator = ' ')) # gem clean sætninger til liste
        
        content.append(' '.join(sentences_text).strip()) # join, strip og append clean sætninger til liste
        #print(f"Antal content: {len(content)}")

    return None

# Test of ftcasetype 

In [12]:
ftcasetype_list = []
itemnum_list =[]

In [60]:
def extract_ftcasetype(xml): 
    soup = bs(xml, 'lxml')
    dagsordenpunkter = soup.find_all("dagsordenpunkt") # liste med dagsordenpunkter
    #print(f"Antal dagsordenpunkter: {len(dagsordenpunkter)}")
    
    for dp in dagsordenpunkter: # find raw xml-sætninger for hvert dagsordenpunkt
        ftcasetype_raw = dp.find_all("ftcasetype") # liste med raw sætninger 
        itemno_raw = dp.find_all("itemno") # liste med raw sætninger
        #print(ftcasetype_raw)
        #print(f"Antal sentences_raw: {len(sentences_raw)}")
        
        
        for sr in ftcasetype_raw: # for hver raw sætning, kør .text af sætningen
            ftcasetype = sr.get_text(separator = ' ') # gem clean sætninger til liste
            #itemno = sr.get_text(separator = ' ')
            ftcasetype_list.append(ftcasetype) # join, strip og append clean sætninger til list
            #itemno_list.append(itemno)
            #print(ftcasetype)
        #print(f"Antal content: {len(content)}")

        for sr in itemno_raw: # for hver raw sætning, kør .text af sætningen
            #ftcasetype = sr.get_text(separator = ' ') # gem clean sætninger til liste
            itemno = sr.get_text(separator = ' ')
            #ftcasetype_list.append(ftcasetype) # join, strip og append clean sætninger til list
            itemno_list.append(itemno)
            print(itemno)
        
    return None

# Sample run

In [66]:
ftcasetype_list = []
itemno_list = []
xml_sample = raw_xml_list[100:101]

In [67]:
for xml in xml_sample: 
    extract_ftcasetype(xml) # FTCaseType

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
23
24
25
26
27
28
30
31
32
33
34
35
36
37
0


In [68]:
len(ftcasetype_list)

36

In [69]:
content =[]
for xml in tqdm.tqdm(xml_sample): 
    extract_content(xml) # content

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.08s/it]


In [70]:
len(content)

37

# Full run

In [15]:
# empty lists (columns)
date = []
content = []
meetingnumber = []
session = []
group = []
location = []
titel = [] # aktivitet
path = []
ftcasetype = []

In [95]:
for xml in tqdm.tqdm(raw_xml_list): 
    extract_metadata(xml)

 27%|█████████████████████▌                                                         | 407/1489 [04:32<12:03,  1.50it/s]


KeyboardInterrupt: 

In [ ]:
for xml in tqdm.tqdm(raw_xml_list): 
    extract_content(xml)

In [ ]:
extract_path(tqdm.tqdm(raw_xml_list))

In [104]:
extract_ftcasetype((raw_xml_list))

AttributeError: 'list' object has no attribute 'startswith'

### title kladdder

In [ ]:
soup = bs(sample1, 'lxml')
test = soup.find("punkttekst").find_all("char")
test

In [ ]:
sample1 = raw_xml_list[0]

In [ ]:
sample1 = raw_xml_list[0]
soup = bs(sample1, 'lxml')

# Make DataFrame

In [22]:
df = pd.DataFrame([date, content, meetingnumber, session, group, location, path, ftitemtype]).transpose()
df.columns = ['date', 'content', 'meetingnumber', 'session', 'group', 'location', 'path', 'ftitemtype']

NameError: name 'ftitemtype' is not defined

In [ ]:
df

# Examining the empty content rows 

# Save df to csv

In [65]:
df.to_csv('FT.csv', index = False)

In [66]:
df = pd.read_csv('FT.csv')